# Extracting and Loading the CSV

In [1]:
# importing libraries

import os
import sys
import pandas as pd
import psycopg2
import pandas as pd

In [2]:
# loading scripts

sys.path.append(os.path.abspath("../scripts/python/"))
from Extractor import Extractor
from Loader import Loader

extractor = Extractor()
loader = Loader()

## Extract Data

In [3]:
# load csv into a dataframe
data = extractor.load_csv("../data/sample.csv")
data.head()

,track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time
0,1; Car; 48.85; 9.770344; 37.977391; 23.737688;...
1,2; Motorcycle; 98.09; 19.839417; 37.977642; 23...
2,3; Motorcycle; 63.80; 18.228752; 37.977997; 23...
3,4; Motorcycle; 145.72; 26.229014; 37.978135; 2...
4,5; Motorcycle; 138.01; 24.841425; 37.978134; 2...


In [4]:
# restructure data frame.
new_df = extractor.restructure(data)
new_df.head()

dataframe successfully created


,track_id,types,traveled_d,avg_speed,trajectory
0,1,Car,48.85,9.770344,"37.977391, 23.737688, 4.9178, 0.0518, -0.0299,..."
1,2,Motorcycle,98.09,19.839417,"37.977642, 23.737400, 16.9759, -0.0361, -0.022..."
2,3,Motorcycle,63.80,18.228752,"37.977997, 23.737264, 20.1906, -0.0795, -0.339..."
3,4,Motorcycle,145.72,26.229014,"37.978135, 23.737072, 2.7555, -0.0302, 0.0948,..."
4,5,Motorcycle,138.01,24.841425,"37.978134, 23.737103, 0.0000, 0.0000, 0.0000, ..."


## Load Into Warehouse

In [5]:
# create connection with wareouse postgres
connection, cursor = loader.connect_to_server(host="localhost", user="warehouse", password="warehouse", dbName="warehouse") #

successfully connected


In [6]:
# create connection with mysql warehouse
connection2, cursor2 = loader.connect_to_mysql_server(host="localhost", user="root", password="mysql", port=42060, dbName="warehouse") #

successfully connected


In [7]:
# load rows

migration = loader.load_from_source(cursor2, "source", 10, "../data/test.csv")
migration.head(5)

/home/bini/anaconda3/envs/py38/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


error: 'CMySQLCursorBuffered' object has no attribute 'cursor'


AttributeError: 'NoneType' object has no attribute 'head'

In [9]:
# create table if it doesn't exist.
loader.create_table(cursor, "../scripts/sql/create_source_table.sql", "warehouse")

table created successfully
table created successfully


In [12]:
# migrate table first.
loader.insert_into_table(cursor, connection, "warehouse", migration, "source")

Data inserted successfully


In [13]:
loader.insert_into_table(cursor, connection, "warehouse", new_df, "source")

Error:  duplicate key value violates unique constraint "source_pkey"
DETAIL:  Key (track_id)=(1) already exists.

Error:  duplicate key value violates unique constraint "source_pkey"
DETAIL:  Key (track_id)=(2) already exists.

Error:  duplicate key value violates unique constraint "source_pkey"
DETAIL:  Key (track_id)=(3) already exists.

Error:  duplicate key value violates unique constraint "source_pkey"
DETAIL:  Key (track_id)=(4) already exists.

Error:  duplicate key value violates unique constraint "source_pkey"
DETAIL:  Key (track_id)=(5) already exists.

Error:  duplicate key value violates unique constraint "source_pkey"
DETAIL:  Key (track_id)=(6) already exists.

Error:  duplicate key value violates unique constraint "source_pkey"
DETAIL:  Key (track_id)=(7) already exists.

Error:  duplicate key value violates unique constraint "source_pkey"
DETAIL:  Key (track_id)=(8) already exists.

Error:  duplicate key value violates unique constraint "source_pkey"
DETAIL:  Key (track

In [12]:
loader.close_connection(cursor=cursor, connection=connection)

connection closed and transaction committed
